In [ ]:
import sys
project_path = r"d:/Download_Github_Desktop/Khoa_Luan_Tot_Nghiep_Agentic_RAG/"
sys.path.append(project_path)
import json 
from source.function.utils_shared import clean_code_fence_safe,fix_json_string
from source.model.extract_model import Bert_Extract
from source.model.generate_model import Gemini
from source.extract.utils_extract import Extract_Information
from source.core.config import Settings
from source.generate.generate import Gemini_Generate
from source.tool.google_search import GoogleSearchTool
from source.core.config import Settings

In [1]:
import sys
project_path = r"D:\DaiHoc\machinelearning\TLCN\DoAnTotNghiep_chat_bot"
sys.path.append(project_path)
import json 
from source.tool.google_search import GoogleSearchTool
from source.core.config import Settings

In [2]:
query="boa nhiêu tuổi đăng kí căn cước công dân ?"


In [3]:
setting=Settings()

In [4]:
google_search_tools=GoogleSearchTool(setting)

In [5]:
result_link=google_search_tools.search(query)

> list các đường link  -> 5 link

In [6]:
print(result_link)

['https://hcmc.vietnam.embassy.gov.au/hchivietnamese/thi-thuc-va-di-tru.html', 'https://vn.usembassy.gov/vi/required-documents/', 'https://xaydungchinhsach.chinhphu.vn/huong-dan-nop-ho-so-de-nghi-cap-the-can-cuoc-cho-tre-em-tu-6-den-duoi-14-tuoi-qua-cong-dvc-11924080407514591.htm', 'https://dichvucong.bocongan.gov.vn/bocongan/bothutuc/tthc?matt=57072', 'https://www.uscis.gov/sites/default/files/document/guides/M-618_v.pdf']


> list các nội dung trang web (5 nội dung)

In [7]:
result_docs=google_search_tools.extract_texts_from_links(result_link)

In [9]:
print(result_docs)

['Thị thực và Di trú\nTổng Lãnh sự quán Úc\nThành phố Hồ Chí Minh, Việt Nam\nSkip to main content\nText size\nEnglish\nTiếng Việt\nToggle navigation\nVề Tổng Lãnh sự quán Úc\nTrang chủ\nVề chúng tôi\nTuyển dụng\nMời Thầu\nDịch vụ\nDịch Vụ Hộ Chiếu\nDịch Vụ Công Chứng\nDịch Vụ Lãnh Sự\nSinh sống tại Việt Nam\nCác dịch vụ khác cho công dân Úc\nKết nối với Úc\nThị thực và Di trú\nDu lịch đến Úc\nDu học Úc\nKinh doanh tại Úc\nGiới thiệu về đất nước Úc\nGiới thiệu về nước Úc\nSự kiện\nHương vị Úc (Taste of Australia) 2025\nTriển Lãm Hương Vị Úc 2025\nTin tức và báo chí\nTin tức từ Đại sứ quán\nĐại Sứ Quán Úc ở Hà Nội\nĐại Sứ Quán Úc ở Hà Nội\nLiên hệ\nLiên hệ\nThị thực và Di trú\nThị thực và Di trú\nEnglish\nI Vietnamese\nThông tin mới nhất\nCảnh giác với các đối tượng lừa đảo nhắm đến người đăng ký tham gia chương trình lựa chọn ngẫu nhiên để nộp hồ sơ xin thị thực Lao động kết hợp Kỳ nghỉ\nBộ Nội Vụ Úc\xa0đã tiếp nhận các báo cáo đáng quan ngại về việc các đối tượng lừa đảo đang nhắm đến 

In [8]:
print(len(result_docs))

5


> Đưa qua Bert 

> tạo model bert và truyền vào để lấy hàm extract thông tin

In [ ]:
bert_model=Bert_Extract(setting)
bert_utils=Extract_Information(bert_model)

> trả ra dưới dạng  {key:value} ở đây key là thứ tự document,value là text sau khi được rút

In [ ]:
result_reduce=bert_utils.predict(result_docs,query)

In [ ]:
print(result_reduce)

> truyền ngữ cảnh đã trích xuất vào gemini

In [ ]:
gemini=Gemini(setting)
gemini_func=Gemini_Generate(gemini,setting)

In [ ]:
result_final=gemini_func.generate_response(query,result_reduce)

In [ ]:
print(result_final)

> gọi 3 hàm này để xử lý

In [ ]:
answer_result= clean_code_fence_safe(result_final)
answer_result= fix_json_string(answer_result)
answer_result= json.loads(answer_result)

In [ ]:
print(answer_result)

In [ ]:
print(answer_result['answer'])

> key này mapping qua link để sắp xếp thứ tự theo cái này , key này nó lấy link 1 nên mapping qua cái list link ban đầu để lấy đường link, rồi hiển thị ưu tiên đầu tiên.Gemini đâu vào là 5 link nhưng việc nó chọn link nào là do bản thân nó.Nói chung tui viết oke hết rồi ô xử lý logic đoạn FE với BE xíu 

In [ ]:
print(answer_result['key'])

In [1]:
import sys
project_path = r"D:/DaiHoc/machinelearning/TLCN/DoAnTotNghiep_chat_bot/"
sys.path.append(project_path)
from source.function.utils_result import RAG
from source.search.utils_search import Qdrant_Utils
from source.rerank.utils_rerank import Rerank_Utils  
from source.model.embedding_model import Sentences_Transformer_Embedding
from source.model.extract_model import Bert_Extract
from source.model.generate_model import Gemini
from source.model.rerank_model import Cohere
from source.data.vectordb.qdrant import Qdrant_Vector
from source.core.config import Settings
from source.model.rerank_model_finetune import RerankModelFinetune
from source.generate.generate import Gemini_Generate
from source.extract.utils_extract import Extract_Information
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score
import cohere
import numpy as np
import pandas as pd
from tqdm import tqdm
setting=Settings()
gemini=Gemini(setting)
print(setting.RERANK)
cohere_api=Cohere(setting)
bert=Bert_Extract(setting)
rerank_model_fintuned=RerankModelFinetune(setting)
sentences_transformer_embedding=Sentences_Transformer_Embedding(setting)
qdrant=Qdrant_Vector(setting,sentences_transformer_embedding)
rerank_Utils=Rerank_Utils(cohere_api,rerank_model_fintuned)
extract_Utils= Extract_Information(bert)
generate_Utils=Gemini_Generate(gemini,setting)
qdrant_Utils=Qdrant_Utils(qdrant, generate_Utils)
rag=RAG(generate_Utils,extract_Utils,qdrant_Utils,rerank_Utils,setting,sentences_transformer_embedding)
import pandas as pd
from tqdm import tqdm
import google.generativeai as genai 
from sklearn.metrics.pairwise import cosine_similarity
import os
from source.tool.utils_search import Utils_Search_Tools
from source.tool.google_search import GoogleSearchTool

hghaan/rerank_model


In [2]:
# Khởi tạo các tools cần thiết cho web search
google_search_tools = GoogleSearchTool(setting)
search_tools = Utils_Search_Tools(setting, generate_Utils, extract_Utils, google_search_tools)

In [4]:
text=search_tools.Search_Docs_From_Tools(query="Bao nhiêu tuổi được phép đăng kí kết hôn ?")

c:\Users\hdang\.virtualenvs\TLCN-KU7o-pax\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'luatsudian.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\hdang\.virtualenvs\TLCN-KU7o-pax\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'thuvienphapluat.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\hdang\.virtualenvs\TLCN-KU7o-pax\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'thuvienphapluat.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:

In [7]:
print(text)

('Độ tuổi được phép đăng ký kết hôn được quy định như sau:\\n\\n*   Nam giới phải từ 20 tuổi trở lên.\\n*   Nữ giới phải từ 18 tuổi trở lên.\\n\\nĐộ tuổi này được tính dựa trên ngày, tháng và năm sinh.\\n\\nVậy, nam từ 20 tuổi và nữ từ 18 tuổi trở lên được phép đăng ký kết hôn.', ['https://luatsudian.com/do-tuoi-dang-ky-ket-hon.html', 'https://thuvienphapluat.vn/chinh-sach-phap-luat-moi/vn/ho-tro-phap-luat/tu-van-phap-luat/61514/huong-dan-xac-dinh-do-tuoi-dang-ky-ket-hon-cua-nam-va-nu-moi-nhat'])
